# Abstractive Text Summarization with Amazon Titan

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Architecture

![](./images/42-text-summarization-2.png)

## Setup

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Configure LangChain with Bedrock

In [ ]:
from langchain.llms.bedrock import Bedrock
modelId = "amazon.titan-tg1-large"
llm = Bedrock(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

## Load the text to summarise

In `letters` directory, you can find a text file of [Amazon's CEO letter to shareholders in 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders). The following cell loads the text file and counts the number of tokens in the file.

In [ ]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

## Split the long text into chunks

The text is too long to fit in the prompt, so we will split it into smaller chunks.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

## How many documents do we now have?

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Summarise chunks using `refine`

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=False)

## Run the summary chain

> ⏰ **Note:** Depending on your number of documents, Bedrock request rate quota, and configured retry settings - the chain below may take some time to run.

In [ ]:
output = ""
try:
    
    output = summary_chain.run(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

## Print the response

In [ ]:
print_ww(output.strip())

## Pre-prepared output

```
Jeff Bezos, the CEO of Amazon, is positive and energized about the company's future despite facing
challenges in 2022. The company grew demand, innovated in its largest businesses, and made
adjustments to its investment decisions while preserving long-term investments. Amazon's success can
be attributed to its ability to adapt to changing market conditions and customer needs. The company
has embraced change and taken steps to streamline its costs, such as shutting down certain
businesses, amending programs, and eliminating 27,000 corporate roles. Amazon is optimistic about
its future prospects and believes that its team is responding to the changes in front of them
effectively.

Amazon has decided to require its corporate employees to return to the office at least three days a
week, starting in May. The company believes that collaborating and inventing is easier and more
effective when done in person, and many of its best inventions have come from serendipitous
interactions in the office. The rising cost to serve in its Stores fulfillment network has been a
critical challenge, and the company has made several changes to improve fulfillment costs and speed
of delivery. During the early part of the pandemic, Amazon's consumer business grew at an
extraordinary clip, doubling the fulfillment center footprint and substantially accelerating
building a last-mile transportation network.

Amazon has made significant structural changes to deliver lower costs and faster speed for many
years to come. These changes include reevaluating how its US fulfillment network was organized and
leveraging its larger fulfillment center footprint to move from a national fulfillment network to a
regionalized network model. Advanced machine learning algorithms are being used to better predict
what customers in various parts of the country will need, and the regional roll out has been
completed. AWS has an $85B annualized revenue run rate and is still early in its adoption curve, but
faces short-term headwinds due to companies being more cautious in spending. AWS is taking a
different tack, focusing on building customer relationships and a business that outlast all of us.
Its sales and support teams are spending much of their time helping customers optimize their AWS
spend so they can better weather this uncertain economy.

AWS faces short-term challenges, but it remains strong due to a robust customer pipeline, active
migrations, and enterprises opting for AWS for its agility, innovation, cost-efficiency, and
security benefits. The company continues to deliver new capabilities rapidly and invests in long-
term inventions, such as its Graviton CPU processors and machine learning training and inference
chips. Despite these challenges, AWS is well-positioned for unusual growth in the next decade.
Amazon Advertising is committed to being the best place for advertisers to build their brands. They
are investing in machine learning to improve their advertising selection algorithms and building
comprehensive, flexible, and durable planning and measurement solutions. Amazon Marketing Cloud
(AMC) is a secure digital environment where advertisers can run custom audience and campaign
analytics in a privacy-safe manner.

The Advertising and AWS teams have collaborated to enable companies to store their data in AWS,
operate securely in AMC, perform analytics in AWS, and activate advertising on Amazon or third-party
publishers through the Amazon Demand-Side Platform. They are also looking at new investment
opportunities, such as expanding from just selling Books to adding categories like Music, Video,
Electronics, and Toys. In 2022, their international consumer segment drove $118B of revenue, with
30% compound annual growth rate (CAGR) in the UK, 26% in Germany, and 21% in Japan. They have also
invested in new international geographies, such as India, Brazil, Mexico, Australia, various
European countries, the Middle East, and parts of Africa. To solve these challenges, they are
working with partners to deliver solutions for customers. Ultimately, this investment in serving a
broader geographical footprint will allow them to help more customers across the world and build a
larger free cash flow-generating consumer business.

Amazon has been expanding its customer offerings across large, unique product retail market
segments, such as grocery. It has built a significant grocery business over nearly 20 years,
offering more than three million items compared to a typical supermarket's 30K. To serve more of its
customers' grocery needs, Amazon needs a broader physical store footprint. Whole Foods Market
pioneered the natural and organic specialty grocery store concept 40 years ago, and Amazon Fresh is
the brand they've been experimenting with for a few years. Amazon Business is an investment where
ecommerce and logistics capabilities position Amazon well to pursue the large market segment. It
launched in 2015 and drives $35B in annualized gross sales, with more than six million active
customers, including 96 of the global Fortune 100 companies.

Amazon has developed Buy with Prime to assist third-party brands and sellers in offering their
products on their websites to Amazon Prime members. It provides merchants with additional benefits
such as Amazon handling product storage, picking, packing, delivery, payment, and returns. It has
increased shopper conversion on third-party shopping sites by 25% on average. Amazon is expanding
internationally, pursuing large retail market segments, and using its unique assets to help
merchants sell more effectively on their websites. In 2003, AWS was a classic example; in 2023,
Amazon Healthcare and Kuiper are potential analogues.

Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills,
and savings for Prime members. However, customers have expressed a strong desire for Amazon to
provide a better alternative to the inefficient and unsatisfying broader healthcare experience. In
July 2022, Amazon announced its acquisition of One Medical, a patient-focused experience with a
digital app and offices in cities across the US. One Medical has relationships with specialty
physicians and works closely with local hospital systems to make seeing specialists easy. Amazon and
One Medical will continue to innovate together to change primary care.

Amazon is creating Kuiper, a low-Earth orbit satellite system to deliver high-quality broadband
internet service to places around the world that don't currently have it. The system will deliver
not only accessibility but affordability, with low-cost antennas that will lower the barriers to
access. The company is preparing to launch two prototype satellites to test the entire end-to-end
communications network this year and plans to be in beta with commercial customers in 2024. Kuiper
represents a large potential opportunity for Amazon, with a large prospective consumer, enterprise,
and government customer base, significant revenue and operating profit potential, and relatively few
companies with the technical and inventive aptitude to go after it.

Amazon is investing heavily in Large Language Models (LLMs) and Generative AI to invent in every
area of its business for many decades to come. Machine learning has been a technology with high
promise for several decades, but it has only been the last five to ten years that it has started to
be used more pervasively by companies. Generative AI is based on very Large Language Models (trained
on up to hundreds of billions of parameters, and growing), across expansive datasets, and has
radically general and broad recall and learning capabilities. AWS is offering the most price-
performant machine learning chips in Trainium and Inferentia so small and large companies can afford
to train and run their LLMs in production. LLMs and Generative AI are going to be a big deal for
customers, our shareholders, and Amazon, and AWS is democratizing this technology so companies of
all sizes can leverage it.

The most important details in this text are the two simple statistics that underline Amazon's
immense future opportunity. While Amazon has a consumer business that's $434B in 2022, the vast
majority of total market segment share in global retail still resides in physical stores (roughly
80%). And, it's a similar story for Global IT spending, where AWS revenue of $80B in 2022, with
about 90% of Global IT spending still on-premises and yet to migrate to the cloud. As these
equations steadily flip, Amazon believes its leading customer experiences, relentless invention,
customer focus, and hard work will result in significant growth in the coming years. Additionally,
Amazon is pursuing other businesses and experiences, all of which are still in their early days. The
CEO is optimistic that Amazon will emerge from this challenging macroeconomic time in a stronger
position than when it entered it and looks forward to working with his teammates to make it so.
```